In [44]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [45]:
df = pd.read_csv('..\\..\\Datasets\\titanic_train.csv')
df.sample()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
553,554,1,3,"Leeni, Mr. Fahim (""Philip Zenni"")",male,22.0,0,0,2620,7.225,NaN,C


# Pipeline
1. Fill all missing values using Column Transformer
2. Perform OHE on Sex, Embarked using Column Transformer
3. Scaling
4. Feature Selection
5. Decision Tree

In [46]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],axis=1, inplace=True)

In [47]:
df.sample()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
385,0,2,male,18.0,0,0,73.5,S


### Train Test Split

In [48]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']), df['Survived'], test_size=0.2, random_state=42)
X_train.sample()
# X_train.shape,y_test.shape


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
423,3,female,28.0,1,1,14.4,S


### Imputation Transformer

In [49]:
trf1 = ColumnTransformer(transformers=[
                          ('impute_age', SimpleImputer(),[2]),
                          ('impute_embarked', SimpleImputer(strategy='most_frequent'),[6])
                        ],remainder='passthrough')

### OHE Transformer

In [50]:
trf2 = ColumnTransformer(transformers=[
                          ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6])
                        ],remainder='passthrough')

### Scaling

In [51]:
trf3 = ColumnTransformer(transformers=[
                          ('scale',MinMaxScaler(), slice(0,10))
                          ])

### Feature Selection

In [52]:
trf4 = SelectKBest(score_func=chi2, k=8)

### Train model

In [53]:
trf5 = DecisionTreeClassifier()

### Creating Pipeline

In [54]:
pipe = Pipeline([
        ('trf1',trf1),
        ('trf2',trf2),
        ('trf3',trf3),
        ('trf4',trf4),
        ('trf5',trf5),
      ])

In [55]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001CC979FE2A0>)),
                ('trf5', DecisionTreeClassifier())])